In [ ]:
!pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement mediapipe (from versions: none)
ERROR: No matching distribution found for mediapipe


In [2]:
import cv2
import mediapipe as mp
import numpy as np
import math
import os  # Para crear carpetas
from google.colab.patches import cv2_imshow  # Importa la función para mostrar imágenes en Colab

def calcular_angulo(p1, p2, p3):
    v1 = np.array([p1.x - p2.x, p1.y - p2.y])
    v2 = np.array([p3.x - p2.x, p3.y - p2.y])

    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)

    if norm_v1 == 0 or norm_v2 == 0:
        return 0  # Evitar división por cero

    angulo_rad = np.arccos(dot_product / (norm_v1 * norm_v2))
    angulo_deg = np.degrees(angulo_rad)
    return angulo_deg

def extraer_frames(ruta_video, carpeta_destino="frames"):
    """Extrae todos los frames del vídeo y los guarda en la carpeta de destino."""
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)

    cap = cv2.VideoCapture(ruta_video)
    if not cap.isOpened():
        print(f"No se pudo abrir el vídeo: {ruta_video}")
        return

    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Vídeo: {ruta_video}, FPS: {frame_rate}, Total Frames: {frame_count}")

    frame_num = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        nombre_archivo = f"{carpeta_destino}/frame_{frame_num:04d}.jpg"
        cv2.imwrite(nombre_archivo, frame)
        frame_num += 1

    cap.release()
    print(f"Se extrajeron {frame_num} frames y se guardaron en '{carpeta_destino}'.")

def analizar_frame(frame, pose):
    """Analiza un solo frame para obtener la pose y varios parámetros."""
    import mediapipe as mp  # Aseguramos que mp esté en el scope
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)
    angulo_codo = None
    distancia_mano_cuerpo = None
    altura_mano = None
    landmarks = None

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # --- Ángulo del Codo ---
        hombro = landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER]
        codo = landmarks[mp.solutions.pose.PoseLandmark.LEFT_ELBOW]
        muneca = landmarks[mp.solutions.pose.PoseLandmark.LEFT_WRIST]
        angulo_codo = calcular_angulo(hombro, codo, muneca)
        cv2.putText(frame, f"Codo: {angulo_codo:.2f} deg", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # --- Distancia Mano-Cuerpo (aproximada) ---
        mano = landmarks[mp.solutions.pose.PoseLandmark.LEFT_WRIST]
        hombro_central = landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER] # Usamos un punto del tronco como referencia
        distancia_mano_cuerpo = math.sqrt((mano.x - hombro_central.x)**2 + (mano.y - hombro_central.y)**2)
        cv2.putText(frame, f"Distancia Mano-Cuerpo: {distancia_mano_cuerpo:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # --- Altura de la Mano (normalizada) ---
        altura_mano = mano.y
        cv2.putText(frame, f"Altura Mano (norm): {altura_mano:.2f}", (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS)

        return frame, angulo_codo, distancia_mano_cuerpo, altura_mano, landmarks

    return frame, None, None, None, None

def analizar_frame_especifico(ruta_video, frame_contacto):
    import mediapipe as mp  # Aseguramos que mp esté en el scope
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=True, model_complexity=2, enable_segmentation=False, min_detection_confidence=0.5)

    cap = cv2.VideoCapture(ruta_video)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_contacto)
    ret, frame = cap.read()
    cap.release()

    if not ret:
        print(f"No se pudo leer el frame {frame_contacto} de {ruta_video}")
        return None, None, None, None, None

    return analizar_frame(frame, pose)

# Rutas de los vídeos
video_pro = '/content/drive/MyDrive/novak/novak.mp4'
video_amateur = '/content/drive/MyDrive/amateur/amateur.mp4'

# Extraer todos los frames de ambos vídeos
extraer_frames(video_pro, "frames_profesional")
extraer_frames(video_amateur, "frames_amateur")

frame_contacto_pro = 145 # Ejemplo, ajustar al frame de contacto real
frame_contacto_ama = 30  # Ejemplo, ajustar al frame de contacto real

resultados_pro = analizar_frame_especifico(video_pro, frame_contacto_pro)
resultados_ama = analizar_frame_especifico(video_amateur, frame_contacto_ama)

if resultados_pro is not None and resultados_ama is not None:
    frame_pro_contacto, angulo_codo_pro, distancia_mc_pro, altura_mano_pro, _ = resultados_pro
    frame_ama_contacto, angulo_codo_ama, distancia_mc_ama, altura_mano_ama, _ = resultados_ama

    cv2_imshow(frame_pro_contacto)  # Solo el frame
    print(f"--- Profesional (Frame {frame_contacto_pro}) ---")
    print(f"Ángulo del codo: {angulo_codo_pro:.2f} grados")
    print(f"Distancia Mano-Cuerpo: {distancia_mc_pro:.2f}")
    print(f"Altura Mano (normalizada): {altura_mano_pro:.2f}")

    cv2_imshow(frame_ama_contacto)    # Solo el frame
    print(f"\n--- Amateur (Frame {frame_contacto_ama}) ---")
    print(f"Ángulo del codo: {angulo_codo_ama:.2f} grados")
    print(f"Distancia Mano-Cuerpo: {distancia_mc_ama:.2f}")
    print(f"Altura Mano (normalizada): {altura_mano_ama:.2f}")

else:
    print("Error al procesar los frames de contacto.")

Output hidden; open in https://colab.research.google.com to view.

In [3]:
import cv2
import mediapipe as mp
import numpy as np
import math
import os  # Para crear carpetas
from google.colab.patches import cv2_imshow  # Importa la función para mostrar imágenes en Colab

def calcular_angulo(p1, p2, p3):
    v1 = np.array([p1.x - p2.x, p1.y - p2.y])
    v2 = np.array([p3.x - p2.x, p3.y - p2.y])

    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)

    if norm_v1 == 0 or norm_v2 == 0:
        return 0  # Evitar división por cero

    angulo_rad = np.arccos(dot_product / (norm_v1 * norm_v2))
    angulo_deg = np.degrees(angulo_rad)
    return angulo_deg

def calcular_distancia(p1, p2):
    return math.sqrt((p1.x - p2.x)**2 + (p1.y - p2.y)**2)

def extraer_frames(ruta_video, carpeta_destino="frames"):
    """Extrae todos los frames del vídeo y los guarda en la carpeta de destino."""
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)

    cap = cv2.VideoCapture(ruta_video)
    if not cap.isOpened():
        print(f"No se pudo abrir el vídeo: {ruta_video}")
        return

    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Vídeo: {ruta_video}, FPS: {frame_rate}, Total Frames: {frame_count}")

    frame_num = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        nombre_archivo = f"{carpeta_destino}/frame_{frame_num:04d}.jpg"
        cv2.imwrite(nombre_archivo, frame)
        frame_num += 1

    cap.release()
    print(f"Se extrajeron {frame_num} frames y se guardaron en '{carpeta_destino}'.")

def analizar_frame(frame, pose):
    """Analiza un solo frame para obtener la pose y varios parámetros."""
    import mediapipe as mp  # Aseguramos que mp esté en el scope
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)
    angulo_codo = None
    angulo_hombro = None
    angulo_rodilla = None
    distancia_mano_hombro_vert = None
    distancia_mano_hombro_horiz = None
    alineacion_hombro_cadera = None
    landmarks = None

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # --- Ángulo del Codo ---
        hombro_izq = landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER]
        codo_izq = landmarks[mp.solutions.pose.PoseLandmark.LEFT_ELBOW]
        muneca_izq = landmarks[mp.solutions.pose.PoseLandmark.LEFT_WRIST]
        angulo_codo = calcular_angulo(hombro_izq, codo_izq, muneca_izq)
        cv2.putText(frame, f"Codo: {angulo_codo:.2f} deg", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # --- Ángulo del Hombro (Aproximado en plano sagital) ---
        hombro_der = landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER]
        angulo_hombro = calcular_angulo(hombro_der, hombro_izq, codo_izq)
        cv2.putText(frame, f"Hombro: {angulo_hombro:.2f} deg", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

        # --- Ángulo de la Rodilla ---
        cadera_izq = landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP]
        rodilla_izq = landmarks[mp.solutions.pose.PoseLandmark.LEFT_KNEE]
        tobillo_izq = landmarks[mp.solutions.pose.PoseLandmark.LEFT_ANKLE]
        angulo_rodilla = calcular_angulo(cadera_izq, rodilla_izq, tobillo_izq)
        cv2.putText(frame, f"Rodilla: {angulo_rodilla:.2f} deg", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

        # --- Distancia Vertical Mano-Hombro ---
        distancia_mano_hombro_vert = abs(muneca_izq.y - hombro_izq.y)
        cv2.putText(frame, f"Vert Mano-Hombro: {distancia_mano_hombro_vert:.2f}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

        # --- Distancia Horizontal Mano-Hombro ---
        distancia_mano_hombro_horiz = abs(muneca_izq.x - hombro_izq.x)
        cv2.putText(frame, f"Horiz Mano-Hombro: {distancia_mano_hombro_horiz:.2f}", (10, 140), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

        # --- Alineación Hombro-Cadera (Diferencia en X) ---
        cadera_izq = landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP]
        alineacion_hombro_cadera = hombro_izq.x - cadera_izq.x
        cv2.putText(frame, f"Hombro-Cadera X Diff: {alineacion_hombro_cadera:.2f}", (10, 160), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 2)

        mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS)

        return frame, angulo_codo, angulo_hombro, angulo_rodilla, distancia_mano_hombro_vert, distancia_mano_hombro_horiz, alineacion_hombro_cadera, landmarks

    return frame, None, None, None, None, None, None, None

def analizar_frame_especifico(ruta_video, frame_contacto, min_detection_confidence=0.5):
    import mediapipe as mp  # Aseguramos que mp esté en el scope
    mp_pose = mp.solutions.pose
    with mp_pose.Pose(static_image_mode=True, model_complexity=2, enable_segmentation=False, min_detection_confidence=min_detection_confidence) as pose:
        cap = cv2.VideoCapture(ruta_video)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_contacto)
        ret, frame = cap.read()
        cap.release()

        if not ret:
            print(f"No se pudo leer el frame {frame_contacto} de {ruta_video}")
            return None, None, None, None, None, None, None, None

        return analizar_frame(frame, pose)

# Rutas de los vídeos
video_pro = '/content/drive/MyDrive/novak/novak.mp4'
video_amateur = '/content/drive/MyDrive/amateur/amateur.mp4'

# Extraer todos los frames de ambos vídeos
extraer_frames(video_pro, "frames_profesional")
extraer_frames(video_amateur, "frames_amateur")

frame_contacto_pro = 145 # Ejemplo, ajustar al frame de contacto real
frame_contacto_ama = 30  # Ejemplo, ajustar al frame de contacto real

# Analizar los frames de contacto con una confianza de detección más baja (ejemplo: 0.3)
resultados_pro = analizar_frame_especifico(video_pro, frame_contacto_pro, min_detection_confidence=0.3)
resultados_ama = analizar_frame_especifico(video_amateur, frame_contacto_ama, min_detection_confidence=0.3)

if resultados_pro is not None and resultados_ama is not None:
    frame_pro_contacto, angulo_codo_pro, angulo_hombro_pro, angulo_rodilla_pro, dist_mano_hom_vert_pro, dist_mano_hom_horiz_pro, al_hom_cad_pro, _ = resultados_pro
    frame_ama_contacto, angulo_codo_ama, angulo_hombro_ama, angulo_rodilla_ama, dist_mano_hom_vert_ama, dist_mano_hom_horiz_ama, al_hom_cad_ama, _ = resultados_ama

    if frame_pro_contacto is not None:
        cv2_imshow(frame_pro_contacto)  # Solo el frame
        print(f"--- Profesional (Frame {frame_contacto_pro}) ---")
        print(f"Ángulo del codo: {angulo_codo_pro:.2f} grados")
        print(f"Ángulo del hombro (aprox): {angulo_hombro_pro:.2f} grados")
        print(f"Ángulo de la rodilla: {angulo_rodilla_pro:.2f} grados")
        print(f"Distancia Vertical Mano-Hombro: {dist_mano_hom_vert_pro:.2f}")
        print(f"Distancia Horizontal Mano-Hombro: {dist_mano_hom_horiz_pro:.2f}")
        print(f"Alineación Hombro-Cadera (X Diff): {al_hom_cad_pro:.2f}")
    else:
        print(f"No se pudo analizar la pose del profesional en el frame {frame_contacto_pro}.")

    if frame_ama_contacto is not None:
        cv2_imshow(frame_ama_contacto)    # Solo el frame
        print(f"\n--- Amateur (Frame {frame_contacto_ama}) ---")
        print(f"Ángulo del codo: {angulo_codo_ama:.2f} grados")
        print(f"Ángulo del hombro (aprox): {angulo_hombro_ama:.2f} grados")
        print(f"Ángulo de la rodilla: {angulo_rodilla_ama:.2f} grados")
        print(f"Distancia Vertical Mano-Hombro: {dist_mano_hom_vert_ama:.2f}")
        print(f"Distancia Horizontal Mano-Hombro: {dist_mano_hom_horiz_ama:.2f}")
        print(f"Alineación Hombro-Cadera (X Diff): {al_hom_cad_ama:.2f}")
    else:
        print(f"No se pudo analizar la pose del amateur en el frame {frame_contacto_ama}.")

else:
    print("Error al procesar los frames de contacto.")


Output hidden; open in https://colab.research.google.com to view.

In [4]:
def analizar_comparacion(pro_data, ama_data):
    """Compara los datos biomecánicos entre el profesional y el amateur."""
    reporte = "--- Reporte de Comparación ---\n\n"

    diferencias = {}

    # Comparar ángulo del codo
    diff_codo = pro_data.get('codo', 0) - ama_data.get('codo', 0)
    reporte += f"Ángulo del codo: Profesional = {pro_data.get('codo', 'N/A'):.2f} grados, Amateur = {ama_data.get('codo', 'N/A'):.2f} grados, Diferencia = {diff_codo:.2f} grados\n"
    diferencias['codo'] = diff_codo

    # Comparar ángulo del hombro
    diff_hombro = pro_data.get('hombro', 0) - ama_data.get('hombro', 0)
    reporte += f"Ángulo del hombro: Profesional = {pro_data.get('hombro', 'N/A'):.2f} grados, Amateur = {ama_data.get('hombro', 'N/A'):.2f} grados, Diferencia = {diff_hombro:.2f} grados\n"
    diferencias['hombro'] = diff_hombro

    # Comparar ángulo de la rodilla
    diff_rodilla = pro_data.get('rodilla', 0) - ama_data.get('rodilla', 0)
    reporte += f"Ángulo de la rodilla: Profesional = {pro_data.get('rodilla', 'N/A'):.2f} grados, Amateur = {ama_data.get('rodilla', 'N/A'):.2f} grados, Diferencia = {diff_rodilla:.2f} grados\n"
    diferencias['rodilla'] = diff_rodilla

    # Comparar distancia vertical mano-hombro
    diff_vert_mano_hombro = pro_data.get('vert_mano_hombro', 0) - ama_data.get('vert_mano_hombro', 0)
    reporte += f"Distancia Vertical Mano-Hombro: Profesional = {pro_data.get('vert_mano_hombro', 'N/A'):.2f}, Amateur = {ama_data.get('vert_mano_hombro', 'N/A'):.2f}, Diferencia = {diff_vert_mano_hombro:.2f}\n"
    diferencias['vert_mano_hombro'] = diff_vert_mano_hombro

    # Comparar distancia horizontal mano-hombro
    diff_horiz_mano_hombro = pro_data.get('horiz_mano_hombro', 0) - ama_data.get('horiz_mano_hombro', 0)
    reporte += f"Distancia Horizontal Mano-Hombro: Profesional = {pro_data.get('horiz_mano_hombro', 'N/A'):.2f}, Amateur = {ama_data.get('horiz_mano_hombro', 'N/A'):.2f}, Diferencia = {diff_horiz_mano_hombro:.2f}\n"
    diferencias['horiz_mano_hombro'] = diff_horiz_mano_hombro

    # Comparar alineación hombro-cadera
    diff_alineacion_hombro_cadera = pro_data.get('hombro_cadera_x', 0) - ama_data.get('hombro_cadera_x', 0)
    reporte += f"Alineación Hombro-Cadera (X Diff): Profesional = {pro_data.get('hombro_cadera_x', 'N/A'):.2f}, Amateur = {ama_data.get('hombro_cadera_x', 'N/A'):.2f}, Diferencia = {diff_alineacion_hombro_cadera:.2f}\n"
    diferencias['alineacion_hombro_cadera'] = diff_alineacion_hombro_cadera

    reporte += "\n--- Análisis de las Diferencias ---\n"
    for param, diff in diferencias.items():
        if abs(diff) > 10 and "angulo" in param:
            reporte += f"Diferencia notable en el ángulo de {param}: {diff:.2f} grados. El profesional tiene {'mayor' if diff > 0 else 'menor'} flexión.\n"
        elif abs(diff) > 0.02 and "distancia" in param:
            reporte += f"Diferencia notable en la {param}: {diff:.2f}. El profesional tiene una {'mayor' if diff > 0 else 'menor'} distancia.\n"
        elif abs(diff) > 0.01 and "alineacion" in param:
            reporte += f"Diferencia notable en la {param}: {diff:.2f}.\n"

    return reporte

def chatbot_interactivo():
    """Crea un chatbot básico para analizar los resultados."""
    print("Bienvenido al Chatbot de Análisis de Revés de Tenis")

    pro_data = {}
    ama_data = {}

    print("\nIngrese los datos del profesional:")
    pro_data['codo'] = float(input("Ángulo del codo: "))
    pro_data['hombro'] = float(input("Ángulo del hombro (aprox): "))
    pro_data['rodilla'] = float(input("Ángulo de la rodilla: "))
    pro_data['vert_mano_hombro'] = float(input("Distancia Vertical Mano-Hombro: "))
    pro_data['horiz_mano_hombro'] = float(input("Distancia Horizontal Mano-Hombro: "))
    pro_data['hombro_cadera_x'] = float(input("Alineación Hombro-Cadera (X Diff): "))

    print("\nIngrese los datos del amateur:")
    ama_data['codo'] = float(input("Ángulo del codo: "))
    ama_data['hombro'] = float(input("Ángulo del hombro (aprox): "))
    ama_data['rodilla'] = float(input("Ángulo de la rodilla: "))
    ama_data['vert_mano_hombro'] = float(input("Distancia Vertical Mano-Hombro: "))
    ama_data['horiz_mano_hombro'] = float(input("Distancia Horizontal Mano-Hombro: "))
    ama_data['hombro_cadera_x'] = float(input("Alineación Hombro-Cadera (X Diff): "))

    reporte = analizar_comparacion(pro_data, ama_data)
    print("\n--- Reporte del Chatbot ---\n")
    print(reporte)

if __name__ == "__main__":
    chatbot_interactivo()

Bienvenido al Chatbot de Análisis de Revés de Tenis

Ingrese los datos del profesional:
Ángulo del codo: 170
Ángulo del hombro (aprox): 75
Ángulo de la rodilla: 14
Distancia Vertical Mano-Hombro: 35
Distancia Horizontal Mano-Hombro: 345
Alineación Hombro-Cadera (X Diff): 345

Ingrese los datos del amateur:
Ángulo del codo: 345
Ángulo del hombro (aprox): 3445
Ángulo de la rodilla: 35
Distancia Vertical Mano-Hombro: 345
Distancia Horizontal Mano-Hombro: 45
Alineación Hombro-Cadera (X Diff): 45

--- Reporte del Chatbot ---

--- Reporte de Comparación ---

Ángulo del codo: Profesional = 170.00 grados, Amateur = 345.00 grados, Diferencia = -175.00 grados
Ángulo del hombro: Profesional = 75.00 grados, Amateur = 3445.00 grados, Diferencia = -3370.00 grados
Ángulo de la rodilla: Profesional = 14.00 grados, Amateur = 35.00 grados, Diferencia = -21.00 grados
Distancia Vertical Mano-Hombro: Profesional = 35.00, Amateur = 345.00, Diferencia = -310.00
Distancia Horizontal Mano-Hombro: Profesional =